In [11]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
from datasets import Dataset
import polars as pl
import json



Mounted at /content/drive


In [12]:
#from datasets import load_dataset, Dataset

sheet_url="/content/drive/MyDrive/Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv"


In [ ]:
'''
Analytical Plan



1. Exploratory Data Analysis (EDA)
Before modeling, perform in-depth EDA to understand:


Data Distribution: How many unique intents, categories, and flags exist? Are they balanced?

Text Analysis:
Length of instruction and response (word count, character count).
Most frequent words/n-grams per intent (using word clouds or bar plots).

Flag Analysis:
TBA

2. Intent Prediction Models (Easy → Complex)
Baseline Models (Easy)

'''


'\nAnalytical Plan\n\n\n\n1. Exploratory Data Analysis (EDA)\nBefore modeling, perform in-depth EDA to understand:\n\n\n\nData Distribution: How many unique intents, categories, and flags exist? Are they balanced?\n\nText Analysis:\nLength of instruction and response (word count, character count).\nMost frequent words/n-grams per intent (using word clouds or bar plots).\nSentiment analysis (if relevant).\nFlag Analysis:\nRelationship between flags and intent (e.g., do certain flags correlate with specific intents?).\nIntent-Category Relationship:\nHierarchical structure? Can category help predict intent?\n2. Intent Prediction Models (Easy → Complex)\nBaseline Models (Easy)\n\n\n\nTF-IDF + Logistic Regression / SVM\n\n\n\nVectorize text (instruction or response) using TF-IDF.\nTrain a simple classifier (Logistic Regression or SVM) to predict intent.\nPros: Easy to implement, interpretable.\nCons: Ignores word order and context.\n\n\n\nTF-IDF + Random Forest / XGBoost\n\n\n\nHandles non-

In [ ]:
df = pl.read_csv(
    sheet_url,
)
respon = df

In [16]:
respon.group_by("intent").agg([
  pl.col("instruction").count().alias("count_of_intent")
])

intent,count_of_intent
str,u32
"""check_invoice""",1000
"""check_cancellation_fee""",950
"""review""",997
"""contact_customer_service""",1000
"""change_order""",997
…,…
"""complaint""",1000
"""newsletter_subscription""",999
"""registration_problems""",999


In [17]:
def analyze_data_distribution(df):
  results = {
      'shape': respon.shape,
      'total_samples': respon.height,
      'intents': {
      'unique_count': respon['intent'].n_unique(),
      'value_counts': respon['intent'].value_counts(sort=True),
      'is_balanced': None
      },
      'categories': {
      'unique_count': respon['category'].n_unique(),
      'value_counts': respon['category'].value_counts(sort=True),
      'is_balanced': None
      },
      'flags': {
      'unique_count': respon['flags'].n_unique(),
      'value_counts': respon['flags'].value_counts(sort=True),
      'is_balanced': None
      }
  }



  # Determine if distributions are balanced (heuristic: no class > 50% of total)
  for col in ['intents', 'categories', 'flags']:
    value_counts = results[col]['value_counts']
    max_percentage = (value_counts['count'].max() / results['total_samples']) * 100
    results[col]['is_balanced'] = max_percentage < 50


  return results



def print_analysis_results(results):
  print(f"Total samples: {results['total_samples']}\n")

  for col in ['intents', 'categories', 'flags']:
    print(f"=== {col.upper()} ===")
    print(f"Unique {col}: {results[col]['unique_count']}")
    print(f"Distribution (top 5):")
    print(results[col]['value_counts'].head(5))
    print(f"Is balanced: {'Yes' if results[col]['is_balanced'] else 'No'}")
    print("\n")


In [18]:
results=analyze_data_distribution(respon)

print_analysis_results(results)

Total samples: 26872

=== INTENTS ===
Unique intents: 27
Distribution (top 5):
shape: (5, 2)
┌──────────────────────────┬───────┐
│ intent                   ┆ count │
│ ---                      ┆ ---   │
│ str                      ┆ u32   │
╞══════════════════════════╪═══════╡
│ check_invoice            ┆ 1000  │
│ complaint                ┆ 1000  │
│ contact_customer_service ┆ 1000  │
│ edit_account             ┆ 1000  │
│ switch_account           ┆ 1000  │
└──────────────────────────┴───────┘
Is balanced: Yes


=== CATEGORIES ===
Unique categories: 11
Distribution (top 5):
shape: (5, 2)
┌──────────┬───────┐
│ category ┆ count │
│ ---      ┆ ---   │
│ str      ┆ u32   │
╞══════════╪═══════╡
│ ACCOUNT  ┆ 5986  │
│ ORDER    ┆ 3988  │
│ REFUND   ┆ 2992  │
│ INVOICE  ┆ 1999  │
│ CONTACT  ┆ 1999  │
└──────────┴───────┘
Is balanced: Yes


=== FLAGS ===
Unique flags: 394
Distribution (top 5):
shape: (5, 2)
┌───────┬───────┐
│ flags ┆ count │
│ ---   ┆ ---   │
│ str   ┆ u32   │
╞═══════╪═════

| Component              | First Block                     | Second Block                                           |
| ---------------------- | ------------------------------- | ------------------------------------------------------ |
| Data analyzed          | `instruction`                   | `response`                                             |
| Output dictionary name | `instruction_ngrams_per_intent` | `response_ngrams_per_intent`                           |
| Print statement labels | "Top Unigrams / Bigrams"        | "Top Unigrams in Responses / Top Bigrams in Responses" |
